# Raphtory Python Client - LOTR DEMO

In this demo we will
* Setup the Python Raphtory client 
* Run an algorithm from python and save the result to a file
* Create a Graph from data in Raphtory
* Pull results from an algorithm in Raphtory that was saved to a pulsar topic 
* Add these results to the Graph
* Visualise the graph with pymotif

### Setup

Please ensure you are running the `raphtory-example-lotr`. 

Either the `SimpleRunner` or `PulsarOutputRunner`

From there please obtain the deployment id, e.g. `raphtory_1172949031` from

```
17:32:15.975 [main] INFO  com.raphtory.client.GraphDeployment - Created Graph object with deployment ID 'raphtory_1172949031'.
```

First we setup the various python libraries we will need

You can install the python client via `pip install raphtory-client`

In [1]:
import raphtoryclient
from pymotif import Motif

#### Create Client

Now we create a client.

This is responsible for two things, 

1) it will be able to interface with the raphtory engine to run algorithms as if we are running via scala

2) be able to retrieve data directly from the pulsar interface

In [2]:
raphtory = raphtoryclient.client(raphtory_deployment_id="raphtory_1172949031")

Creating RaphtoryClient object...
Created.
Setting up Java gateway...
Java gateway connected.
Creating Raphtory java object...
Created Raphtory java object.


### Running a algorithm 

To run an algorithm we must first import the java path it belongs to. 
In our case we want to run the Connected components algorithms. 

We will import this into our local client, so that our code is able to call them.

In [17]:
raphtory.java_import("com.raphtory.algorithms.generic.ConnectedComponents")
connectedComponentsAlgorithm = raphtory.java().ConnectedComponents

We also want to write the result of this to a file, so we can use the FileOutputFormat class to do so. 

We first import the class as follows

In [18]:
raphtory.java_import("com.raphtory.output.FileOutputFormat")
fileOutputFormat = raphtory.java().FileOutputFormat

In this example, 

- we are running the algorithm at time 32674 (which is the latest point in our graph). 
- with the `past()` function, which will ensure the graph takes into account all the history from and including 32674
- We 'execute' the connected components algorithm 
- Finally we write this to a file

We can now run the algorithm as follows. 

In [19]:
queryHandler = raphtory.graph\
    .at(32674)\
    .past()\
    .execute(connectedComponentsAlgorithm())\
    .writeTo(fileOutputFormat.apply("/tmp/pythonCC"), "pythonCC")

If we now investigate the `/tmp/pythonCC` file we will see that 
it contains our results. 

You can check the status of this algorithm 
in the java terminal where your raphtory instance is running or you can run the following.

In [21]:
queryHandler.isJobDone()

True

## Exporting data 

Previously we ran an algorithm that wrote our data to a file. 

Now we will look at exporting results from an algorithm that was saved to Pulsar topics. 

This portion of the tutorial requires that you are running the `raphtory-example-lotr/.../PulsarOutputRunner`

We will create the readers which read from topics.

If the connection fails, the code with automatically retry.

This can occur when you have not closed previous pulsar connections.

In this case we are reading the topics: EdgeList, PageRank

Note: Prior to this you should have already run these algorithms in Raphtory.

In [3]:
edgeListReader = raphtory.createReader("EdgeList", subscription_name='edgelist_reader')
pageRankReader = raphtory.createReader("PageRank", subscription_name='pagerank_reader')

2022-04-27 17:39:11.063 INFO  [0x10ff18e00] ConnectionPool:84 | Created connection for pulsar://127.0.0.1:6650
2022-04-27 17:39:11.064 INFO  [0x700010b75000] ClientConnection:372 | [127.0.0.1:56076 -> 127.0.0.1:6650] Connected to broker
2022-04-27 17:39:11.065 INFO  [0x700010b75000] HandlerBase:55 | [persistent://public/default/EdgeList, reader- 67df90f3e, 0] Getting connection from pool
2022-04-27 17:39:11.066 INFO  [0x700010b75000] ConnectionPool:84 | Created connection for pulsar://localhost:6650
2022-04-27 17:39:11.066 INFO  [0x700010b75000] ClientConnection:374 | [127.0.0.1:56077 -> 127.0.0.1:6650] Connected to broker through proxy. Logical broker: pulsar://localhost:6650
2022-04-27 17:39:11.072 INFO  [0x700010b75000] ConsumerImpl:220 | [persistent://public/default/EdgeList, reader- 67df90f3e, 0] Created consumer on broker [127.0.0.1:56077 -> 127.0.0.1:6650] 
Connected to topic: EdgeList
2022-04-27 17:39:11.073 INFO  [0x700010b75000] HandlerBase:55 | [persistent://public/default/P

### Obtain dataframes

Now we will run the getDataframe function to retrieve results as dataframes.

In [4]:
df_edge = raphtory.getDataframe(edgeListReader)

Obtaining dataframe...

Converting to columns...
Completed.


In [5]:
df_edge

,0,1,2
0,30000,Hador,Húrin
1,30000,Ungoliant,Shelob
2,30000,Orophin,Rúmil
3,30000,Arwen,Celebrían
4,30000,Arwen,Frodo
...,...,...,...
533,30000,Elendil,Boromir
534,30000,Elendil,Aragorn
535,30000,Elendil,Elessar
536,30000,Elessar,Halbarad


In [6]:
df_page = raphtory.getDataframe(pageRankReader)

Obtaining dataframe...

Converting to columns...
Completed.


In [7]:
df_page

,0,1,2,3
0,20000,10000,Balin,0.15000000000000002
1,20000,10000,Meriadoc,0.4199991666063432
2,20000,10000,Isildur,0.15000000000000002
3,20000,10000,Amroth,0.2300710908862701
4,20000,10000,Celebrían,0.2775
...,...,...,...,...
150,30000,500,Faramir,0.18028125000000003
151,30000,500,Gollum,0.47526644531250006
152,30000,500,Sam,0.21375000000000002
153,30000,500,Gorbag,0.15000000000000002


### Create a graph

Next we create a graph by pulling the edge list from Raphtory.

In this case we would like to create a graph from the LOTR dataset.

So we run the `createGraphFromEdgeList` method on the `EdgeList` dataframe.

In [8]:
G = raphtory.createGraphFromEdgeList(df_edge, isMultiGraph=False)
G.number_of_nodes(), G.number_of_edges()

Creating graph...
Done.


(124, 538)

### Adding properties to our graph
Now we merge these as node properties into our graph

In [9]:
raphtory.add_node_attributes(G, [df_page], ['PageRank'])

### Visualisation

Finally we plot the graph with an open source visualisation tool .

In [8]:
motif_nx = Motif(nx_graph=G, title='NetworkX')
motif_nx.plot()